In [1]:
import numpy as np,pandas as pd,seaborn as sns

In [2]:
import plotly.express as px,warnings
warnings.filterwarnings('ignore')

In [4]:
df=pd.read_csv('EasyVisa.csv')

In [7]:
features_with_na=[i for i in df.columns if df[i].isnull().sum()>=1]
for i in features_with_na:
    print(i,np.round(df[i].isnull().mean()*100,5),' %')

In [8]:
features_with_na

[]

In [9]:
df.drop('case_id',axis=1,inplace=True)

In [10]:
num_features=[i for i in df.columns if df[i].dtype!='O']
cat_features=[j for j in df.columns if df[j].dtype=='O']
print('we have {} numerical : {}'.format(len(num_features),num_features))
print('we have {} categorical : {}'.format(len(cat_features),cat_features))

we have 3 numerical : ['no_of_employees', 'yr_of_estab', 'prevailing_wage']
we have 8 categorical : ['continent', 'education_of_employee', 'has_job_experience', 'requires_job_training', 'region_of_employment', 'unit_of_wage', 'full_time_position', 'case_status']


In [31]:
x=df.drop('case_status',axis=1)
y=df['case_status']

In [12]:
y=np.where(y=='Denied',1,0)

In [13]:
y

array([1, 0, 1, ..., 0, 0, 0])

In [16]:
df['full_time_position'].value_counts()

full_time_position
Y    22773
N     2707
Name: count, dtype: int64

In [23]:
from datetime import date
  
# creating the date object of today's date
todays_date = date.today()
current_year= todays_date.year

In [24]:
df['company_age'] = current_year-df['yr_of_estab']


In [25]:
df.columns

Index(['continent', 'education_of_employee', 'has_job_experience',
       'requires_job_training', 'no_of_employees', 'yr_of_estab',
       'region_of_employment', 'prevailing_wage', 'unit_of_wage',
       'full_time_position', 'case_status', 'company_age'],
      dtype='object')

In [26]:
num_features = list(x.select_dtypes(exclude="object").columns)
num_features

['no_of_employees', 'yr_of_estab', 'prevailing_wage']

In [27]:
or_columns = ['has_job_experience','requires_job_training','full_time_position','education_of_employee']
oh_columns = ['continent','unit_of_wage','region_of_employment']
transform_columns= ['no_of_employees','company_age']

from sklearn.preprocessing import OneHotEncoder, StandardScaler,OrdinalEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
ordinal_encoder = OrdinalEncoder()

transform_pipe = Pipeline(steps=[
    ('transformer', PowerTransformer(method='yeo-johnson'))
])

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, oh_columns),
        ("Ordinal_Encoder", ordinal_encoder, or_columns),
        ("Transformer", transform_pipe, transform_columns),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)

In [32]:
x

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,company_age
0,Asia,High School,N,N,14513,2007,West,592.2029,Hour,Y,18
1,Asia,Master's,Y,N,2412,2002,Northeast,83425.6500,Year,Y,23
2,Asia,Bachelor's,N,Y,44444,2008,West,122996.8600,Year,Y,17
3,Asia,Bachelor's,N,N,98,1897,West,83434.0300,Year,Y,128
4,Africa,Master's,Y,N,1082,2005,South,149907.3900,Year,Y,20
...,...,...,...,...,...,...,...,...,...,...,...
25475,Asia,Bachelor's,Y,Y,2601,2008,South,77092.5700,Year,Y,17
25476,Asia,High School,Y,N,3274,2006,Northeast,279174.7900,Year,Y,19
25477,Asia,Master's,Y,N,1121,1910,South,146298.8500,Year,N,115
25478,Asia,Master's,Y,Y,1918,1887,West,86154.7700,Year,Y,138


In [33]:
x=preprocessor.fit_transform(x)

In [34]:
from imblearn.combine import SMOTETomek, SMOTEENN

# Resampling the minority class. The strategy can be changed as required.
smt = SMOTEENN(random_state=42,sampling_strategy='minority' )
# Fit the model to generate the data.
x_res, y_res = smt.fit_resample(x, y)

In [35]:
from sklearn.model_selection import  train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_res,y_res,test_size=0.2,random_state=42)
x_train.shape, x_test.shape

((13629, 24), (3408, 24))

In [37]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve 
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [39]:
models = {
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Logistic Regression": LogisticRegression(),
     "K-Neighbors Classifier": KNeighborsClassifier(),
    "XGBClassifier": XGBClassifier(), 
     "CatBoosting Classifier": CatBoostClassifier(verbose=False),
     "Support Vector Classifier": SVC(),
    "AdaBoost Classifier": AdaBoostClassifier()

}

In [44]:
# def evaluate_clf(true, predicted):
#     acc = accuracy_score(true, predicted) # Calculate Accuracy
#     f1 = f1_score(true, predicted) # Calculate F1-score
#     precision = precision_score(true, predicted) # Calculate Precision
#     recall = recall_score(true, predicted)  # Calculate Recall
#     roc_auc = roc_auc_score(true, predicted) #Calculate Roc
#     return acc, f1 , precision, recall, roc_auc

In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.exceptions import NotFittedError

# Define the positive class label string.
# This should match one of the actual string labels in your 'y' data.
POSITIVE_CLASS_LABEL = 'Certified'

def evaluate_clf(y_true, y_pred, y_pred_proba=None, pos_label_encoded=1):
    '''
    Evaluates classification metrics.

    Args:
        y_true (array-like): True labels (expected to be numerical, e.g., 0 or 1).
        y_pred (array-like): Predicted hard labels (expected to be numerical, e.g., 0 or 1).
        y_pred_proba (array-like, optional): Predicted probabilities for the positive class.
                                             Used for ROC AUC calculation. Defaults to None.
        pos_label_encoded (int): The numerical label representing the positive class
                                 (e.g., 1, after LabelEncoding). Defaults to 1.

    Returns:
        tuple: A tuple containing accuracy, F1 score, precision, recall, and ROC AUC score.
    '''
    # Accuracy score does not take 'pos_label'
    accuracy = accuracy_score(y_true, y_pred)

    # For F1, Precision, Recall, use the numerical pos_label_encoded
    f1 = f1_score(y_true, y_pred, pos_label=pos_label_encoded, average='binary')
    precision = precision_score(y_true, y_pred, pos_label=pos_label_encoded, average='binary')
    recall = recall_score(y_true, y_pred, pos_label=pos_label_encoded, average='binary')

    rocauc_score_val = 0.0 # Initialize ROC AUC score

    if y_pred_proba is not None:
        # If probabilities are provided, use them for ROC AUC.
        # y_true is already expected to be numerical (0/1) here.
        rocauc_score_val = roc_auc_score(y_true, y_pred_proba)
    else:
        # If probabilities are not available, use hard predictions.
        # Since y_true and y_pred are already 0/1 numerical, roc_auc_score can handle it.
        rocauc_score_val = roc_auc_score(y_true, y_pred)

    return accuracy, f1, precision, recall, rocauc_score_val

def evaluate_models(x, y, models):
    '''
    This function takes in X (features), y (target), and a dictionary of models as input.
    It performs train-test split, iterates through the given model dictionary,
    and evaluates various classification metrics for each model.

    Args:
        x (DataFrame or array-like): Features dataset.
        y (Series or array-like): Target variable (can be string labels).
        models (dict): A dictionary where keys are model names (strings) and values
                       are instantiated scikit-learn classifier models.

    Returns:
        pd.DataFrame: A DataFrame containing a report of all models' metrics,
                      sorted by accuracy in descending order.
    '''
    # Initialize LabelEncoder to convert string labels to numerical (0 and 1)
    le = LabelEncoder()

    # Fit and transform y BEFORE splitting the data.
    # This ensures that both train and test sets have numerical labels (0 and 1).
    y_encoded = le.fit_transform(y)

    # Determine which numerical label corresponds to the defined POSITIVE_CLASS_LABEL.
    # LabelEncoder assigns integers based on alphabetical order by default.
    pos_label_mapping = {label: idx for idx, label in enumerate(le.classes_)}

    if POSITIVE_CLASS_LABEL not in pos_label_mapping:
        raise ValueError(
            f"The specified POSITIVE_CLASS_LABEL '{POSITIVE_CLASS_LABEL}' "
            f"was not found in the unique values of 'y': {le.classes_}"
        )

    # Store the numerical representation of your positive class
    NUMERIC_POSITIVE_LABEL = pos_label_mapping[POSITIVE_CLASS_LABEL]
    # The other label (for binary classification)
    # NUMERIC_NEGATIVE_LABEL = 1 - NUMERIC_POSITIVE_LABEL # Not strictly needed for metrics but good for understanding

    # Separate dataset into train and test sets using the encoded target variable
    X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
        x, y_encoded, test_size=0.2, random_state=42
    )

    # Lists to store model evaluation results
    models_list = []
    accuracy_list = []
    auc_list = [] # Renamed from 'auc' to avoid conflict if 'auc' was used elsewhere
    f1_scores = []
    precision_scores = []
    recall_scores = []

    # Iterate through each model in the provided dictionary
    for model_name, model in models.items(): # Use .items() for cleaner iteration
        print(f"--- Evaluating model: {model_name} ---")

        try:
            # Train the model using the encoded training data
            model.fit(X_train, y_train_encoded)

            # Make predictions (these will also be numerical: 0 or 1)
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)

            # Get probabilities for ROC AUC if the model supports it
            y_train_pred_proba = None
            y_test_pred_proba = None
            if hasattr(model, 'predict_proba'):
                try:
                    # Find the index of the positive class within the model's output classes.
                    # This is crucial as model.classes_ order might vary.
                    positive_class_proba_idx = list(model.classes_).index(NUMERIC_POSITIVE_LABEL)
                    y_train_pred_proba = model.predict_proba(X_train)[:, positive_class_proba_idx]
                    y_test_pred_proba = model.predict_proba(X_test)[:, positive_class_proba_idx]
                except ValueError:
                    print(f"Warning: Numeric positive label '{NUMERIC_POSITIVE_LABEL}' "
                          f"not found in model.classes_ for {model_name}. "
                          f"ROC AUC will be calculated with hard predictions.")
                except NotFittedError:
                    print(f"Warning: Model {model_name} not fitted correctly for predict_proba. "
                          f"ROC AUC will be calculated with hard predictions.")
                    pass # Fallback to calculating ROC AUC with hard predictions

            # Evaluate training set performance
            model_train_accuracy, model_train_f1, model_train_precision,\
            model_train_recall, model_train_rocauc_score = evaluate_clf(
                y_train_encoded, y_train_pred, y_train_pred_proba, pos_label_encoded=NUMERIC_POSITIVE_LABEL
            )

            # Evaluate test set performance
            model_test_accuracy, model_test_f1, model_test_precision,\
            model_test_recall, model_test_rocauc_score = evaluate_clf(
                y_test_encoded, y_test_pred, y_test_pred_proba, pos_label_encoded=NUMERIC_POSITIVE_LABEL
            )

            # Append results to lists ONLY if all calculations were successful for this model
            models_list.append(model_name)
            accuracy_list.append(model_test_accuracy)
            f1_scores.append(model_test_f1)
            precision_scores.append(model_test_precision)
            recall_scores.append(model_test_recall)
            auc_list.append(model_test_rocauc_score)

            # Print performance metrics for the training set
            print('Model performance for Training set:')
            print(f"- Accuracy: {model_train_accuracy:.4f}")
            print(f"- F1 score: {model_train_f1:.4f}")
            print(f"- Precision: {model_train_precision:.4f}")
            print(f"- Recall: {model_train_recall:.4f}")
            print(f"- Roc Auc Score: {model_train_rocauc_score:.4f}")

            print('----------------------------------')

            # Print performance metrics for the test set
            print('Model performance for Test set:')
            print(f"- Accuracy: {model_test_accuracy:.4f}")
            print(f"- F1 score: {model_test_f1:.4f}")
            print(f"- Precision: {model_test_precision:.4f}")
            print(f"- Recall: {model_test_recall:.4f}")
            print(f"- Roc Auc Score: {model_test_rocauc_score:.4f}")
            print('='*35)
            print('\n')

        except Exception as e:
            # If any error occurs for a model, print an error message and skip to the next model
            print(f"Error evaluating model {model_name}: {e}")
            print("Skipping this model and continuing with the next.")
            print('='*35)
            print('\n')
            # Do NOT append anything for this failed model to maintain list consistency
            continue # Move to the next model in the loop

    # Create the final report DataFrame from the collected lists
    report_data = {
        'Model Name': models_list,
        'Accuracy': accuracy_list,
        'F1 Score': f1_scores,
        'Precision': precision_scores,
        'Recall': recall_scores,
        'ROC AUC': auc_list
    }
    report = pd.DataFrame(report_data).sort_values(by=['Accuracy'], ascending=False)

    return report

# --- Example Usage ---
if __name__ == "__main__":
    # Create some dummy data for demonstration
    # X (features) - numerical data
    data = {
        'feature1': [10, 20, 15, 25, 30, 12, 18, 22, 28, 35, 11, 16, 21, 26, 31, 13, 19, 23, 29, 34],
        'feature2': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
        'feature3': [5, 8, 6, 9, 7, 5, 8, 6, 9, 7, 5, 8, 6, 9, 7, 5, 8, 6, 9, 7]
    }
    X = pd.DataFrame(data)

    # y (target) - string labels ('Certified' or 'Denied')
    y_labels = ['Certified', 'Denied', 'Certified', 'Denied', 'Certified',
                'Denied', 'Certified', 'Denied', 'Certified', 'Denied',
                'Certified', 'Denied', 'Certified', 'Denied', 'Certified',
                'Denied', 'Certified', 'Denied', 'Certified', 'Denied']
    y = pd.Series(y_labels)

    # Define a dictionary of models to evaluate
    # Ensure your models are instantiated correctly
    models = {
        'Logistic Regression': LogisticRegression(random_state=42, solver='liblinear'),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        # Add more models here, e.g., 'RandomForestClassifier': RandomForestClassifier(random_state=42)
    }

    # Evaluate the models
    evaluation_report = evaluate_models(X, y, models)

    # Print the final report
    print("\n=== Model Evaluation Report ===")
    print(evaluation_report)

    # Example of a model that might cause an error for demonstration (e.g., a custom broken model)
    # class BrokenModel:
    #     def fit(self, X, y):
    #         raise Exception("This model intentionally fails during fitting!")
    #     def predict(self

--- Evaluating model: Logistic Regression ---
Model performance for Training set:
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 0.0000
----------------------------------
Model performance for Test set:
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 0.0000


--- Evaluating model: Decision Tree ---
Model performance for Training set:
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 0.0000
----------------------------------
Model performance for Test set:
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 0.0000



=== Model Evaluation Report ===
            Model Name  Accuracy  F1 Score  Precision  Recall  ROC AUC
0  Logistic Regression       1.0       1.0        1.0     1.0      0.0
1        Decision Tree       1.0       1.0        1.0     1.0      0.0


In [56]:
base_model_report =evaluate_models(x=x_res, y=y_res, models=models)

--- Evaluating model: Logistic Regression ---
Model performance for Training set:
- Accuracy: 0.7385
- F1 score: 0.7226
- Precision: 0.7071
- Recall: 0.7387
- Roc Auc Score: 0.1722
----------------------------------
Model performance for Test set:
- Accuracy: 0.7256
- F1 score: 0.7081
- Precision: 0.6936
- Recall: 0.7232
- Roc Auc Score: 0.1834


--- Evaluating model: Decision Tree ---
Model performance for Training set:
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 0.0000
----------------------------------
Model performance for Test set:
- Accuracy: 0.9263
- F1 score: 0.9193
- Precision: 0.9273
- Recall: 0.9114
- Roc Auc Score: 0.0748


